In [1]:
%matplotlib inline

In [2]:
from __future__ import annotations

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

import pyvista as pv
from pyvista import examples
import pyvistaqt as pvqt
import cv2

np.set_printoptions(linewidth=500)

In [9]:
N_COLUMNS=12
N_ROWS=12

data = np.zeros((N_ROWS, N_COLUMNS))

data[5][5] = 100

data[5][6] = 100
data[5][7] = 100

data[10][5] = 100

data[8][10] = 100



data

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., 100., 100., 100.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [21]:
apple = pv.read('./apple.obj')
tex = pv.read_texture('./texture-2.jpg')

pl = pv.Plotter()
pl.add_mesh(apple, texture=tex)

points = np.load("apple_points_12_12.npy")

for point in points:
    sphere = pv.Sphere(center=point, radius=0.0005)
    pl.add_mesh(sphere, style='points', color='blue')
pl.show()




Widget(value='<iframe src="http://localhost:59871/index.html?ui=P_0x29e93c380_14&reconnect=auto" class="pyvist…

In [23]:
from scipy.spatial import KDTree


k = 5

kdtree = KDTree(points)

apple['nearest_point'] = kdtree.query(apple.points)[1]
apple['nearest_points'] = kdtree.query(apple.points, k=k)[1]
apple['nearest_points_dist'] = kdtree.query(apple.points, k=k)[0]

data_flatten = data.flatten()

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / apple['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2

# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / np.sum(squared_reciprocal_distances, axis=1, keepdims=True)


In [25]:


# Step 4: Apply the weights to compute the weighted sum
apple['pressure'] = np.sum(data_flatten[apple['nearest_points']] * normalized_weights, axis=1)

# apple['pressure'] = np.where(apple['pressure'] > 50, 50, apple['pressure'])

pl = pv.Plotter()
pl.add_mesh(apple, scalars='pressure', cmap='RdPu')
for point in points:
    sphere = pv.Sphere(center=point, radius=0.0005)
    pl.add_mesh(sphere, style='points', color='blue')

pl.show()

Widget(value='<iframe src="http://localhost:59871/index.html?ui=P_0x3113f3230_16&reconnect=auto" class="pyvist…